# Simple Logistic Regression Interpretability - Model Specific and Local --> Reason Codes

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np                   # array, vector, matrix calculations
import pandas as pd                  # DataFrame handling

import statsmodels.api as sm         # standard LR - gives feature significance

from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import Lasso

from sklearn.metrics import classification_report, confusion_matrix, auc, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV

import xgboost as xgb                # gradient boosting machines (GBMs)

import matplotlib.pyplot as plt      # plotting

pd.options.display.max_columns = 999 # enable display of all columns in notebook

# enables display of plots in notebook
%matplotlib inline        

## 1. Download, explore, and prepare UCI credit card default data

UCI credit card default data: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

The UCI credit card default data contains demographic and payment information about credit card customers in Taiwan in the year 2005. The data set contains 23 input variables: 

* **`LIMIT_BAL`**: Amount of given credit (NT dollar)
* **`SEX`**: 1 = male; 2 = female
* **`EDUCATION`**: 1 = graduate school; 2 = university; 3 = high school; 4 = others 
* **`MARRIAGE`**: 1 = married; 2 = single; 3 = others
* **`AGE`**: Age in years 
* **`PAY_0`, `PAY_2` - `PAY_6`**: History of past payment; `PAY_0` = the repayment status in September, 2005; `PAY_2` = the repayment status in August, 2005; ...; `PAY_6` = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; ...; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
* **`BILL_AMT1` - `BILL_AMT6`**: Amount of bill statement (NT dollar). `BILL_AMNT1` = amount of bill statement in September, 2005; `BILL_AMT2` = amount of bill statement in August, 2005; ...; `BILL_AMT6` = amount of bill statement in April, 2005. 
* **`PAY_AMT1` - `PAY_AMT6`**: Amount of previous payment (NT dollar). `PAY_AMT1` = amount paid in September, 2005; `PAY_AMT2` = amount paid in August, 2005; ...; `PAY_AMT6` = amount paid in April, 2005. 

These 23 input variables are used to predict the target variable, whether or not a customer defaulted on their credit card bill in late 2005. 

### All Input variables are numeric and will be used as is (no transformations) for simplicity/interpretability purposes. This should not be the approach if maximizing model performance is the goal.

#### Import data and clean
The credit card default data is available as an `.csv` file. Pandas reads `.csv` files automatically, so it's used to load the credit card default data and give the prediction target a shorter name: `DEFAULT_NEXT_MONTH`. 

In [2]:
# import csv file
path = './default_of_credit_card_clients.csv'
data = pd.read_csv(path) 

# remove spaces from target column name 
data = data.rename(columns={'default payment next month': 'DEFAULT_NEXT_MONTH'}) 

print("Data dimensions:", data.shape)
print(data.head(5))
print(data.dtypes)

Data dimensions: (30000, 24)
   LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0      20000    2          2         1   24      2      2     -1     -1   
1     120000    2          2         2   26     -1      2      0      0   
2      90000    2          2         2   34      0      0      0      0   
3      50000    2          2         1   37      0      0      0      0   
4      50000    1          2         1   57     -1      0     -1      0   

   PAY_5  PAY_6  BILL_AMT1  BILL_AMT2  BILL_AMT3  BILL_AMT4  BILL_AMT5  \
0     -2     -2       3913       3102        689          0          0   
1      0      2       2682       1725       2682       3272       3455   
2      0      0      29239      14027      13559      14331      14948   
3      0      0      46990      48233      49291      28314      28959   
4      0      0       8617       5670      35835      20940      19146   

   BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  \

In [3]:
# assign target and inputs for GBM
y = 'DEFAULT_NEXT_MONTH'
X = [name for name in data.columns if name not in [y, 'ID']]
print('y =', y)
print('X =', X)

data[X + [y]].describe() # display descriptive statistics for all columns

y = DEFAULT_NEXT_MONTH
X = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT_NEXT_MONTH
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [4]:
# check for missing values - none
data.isnull().values.any()

False

In [5]:
# some features are very correlated, let's check the corr matrix
data[X + [y]].corr()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT_NEXT_MONTH
LIMIT_BAL,1.000000,0.024755,-0.219161,-0.108139,0.144713,-0.271214,-0.296382,-0.286123,-0.267460,-0.249411,-0.235195,0.285430,0.278314,0.283236,0.293988,0.295562,0.290389,0.195236,0.178408,0.210167,0.203242,0.217202,0.219595,-0.153520
SEX,0.024755,1.000000,0.014232,-0.031389,-0.090874,-0.057643,-0.070771,-0.066096,-0.060173,-0.055064,-0.044008,-0.033642,-0.031183,-0.024563,-0.021880,-0.017005,-0.016733,-0.000242,-0.001391,-0.008597,-0.002229,-0.001667,-0.002766,-0.039961
EDUCATION,-0.219161,0.014232,1.000000,-0.143464,0.175061,0.105364,0.121566,0.114025,0.108793,0.097520,0.082316,0.023581,0.018749,0.013002,-0.000451,-0.007567,-0.009099,-0.037456,-0.030038,-0.039943,-0.038218,-0.040358,-0.037200,0.028006
MARRIAGE,-0.108139,-0.031389,-0.143464,1.000000,-0.414170,0.019917,0.024199,0.032688,0.033122,0.035629,0.034345,-0.023472,-0.021602,-0.024909,-0.023344,-0.025393,-0.021207,-0.005979,-0.008093,-0.003541,-0.012659,-0.001205,-0.006641,-0.024339
AGE,0.144713,-0.090874,0.175061,-0.414170,1.000000,-0.039447,-0.050148,-0.053048,-0.049722,-0.053826,-0.048773,0.056239,0.054283,0.053710,0.051353,0.049345,0.047613,0.026147,0.021785,0.029247,0.021379,0.022850,0.019478,0.013890
PAY_0,-0.271214,-0.057643,0.105364,0.019917,-0.039447,1.000000,0.672164,0.574245,0.538841,0.509426,0.474553,0.187068,0.189859,0.179785,0.179125,0.180635,0.176980,-0.079269,-0.070101,-0.070561,-0.064005,-0.058190,-0.058673,0.324794
PAY_2,-0.296382,-0.070771,0.121566,0.024199,-0.050148,0.672164,1.000000,0.766552,0.662067,0.622780,0.575501,0.234887,0.235257,0.224146,0.222237,0.221348,0.219403,-0.080701,-0.058990,-0.055901,-0.046858,-0.037093,-0.036500,0.263551
PAY_3,-0.286123,-0.066096,0.114025,0.032688,-0.053048,0.574245,0.766552,1.000000,0.777359,0.686775,0.632684,0.208473,0.237295,0.227494,0.227202,0.225145,0.222327,0.001295,-0.066793,-0.053311,-0.046067,-0.035863,-0.035861,0.235253
PAY_4,-0.267460,-0.060173,0.108793,0.033122,-0.049722,0.538841,0.662067,0.777359,1.000000,0.819835,0.716449,0.202812,0.225816,0.244983,0.245917,0.242902,0.239154,-0.009362,-0.001944,-0.069235,-0.043461,-0.033590,-0.026565,0.216614
PAY_5,-0.249411,-0.055064,0.097520,0.035629,-0.053826,0.509426,0.622780,0.686775,0.819835,1.000000,0.816900,0.206684,0.226913,0.243335,0.271915,0.269783,0.262509,-0.006089,-0.003191,0.009062,-0.058299,-0.033337,-0.023027,0.204149


### Split into Train/Test randomly

In [6]:
np.random.seed(12345) # set random seed for reproducibility
split_ratio = 0.8     # 80%/20% train/test split

# execute split
split = np.random.rand(len(data)) < split_ratio
train = data[split]
test = data[~split]

# summarize split
print('Train data rows = %d, columns = %d' % (train.shape[0], train.shape[1]))
print('Test data rows = %d, columns = %d' % (test.shape[0], test.shape[1]))

Train data rows = 23918, columns = 24
Test data rows = 6082, columns = 24


In [7]:
# check for coefficient significance via standard LR
logit_model=sm.Logit(train[y],train[X])
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.464301
         Iterations 7
                          Results: Logit
Model:              Logit              Pseudo R-squared: 0.120     
Dependent Variable: DEFAULT_NEXT_MONTH AIC:              22256.3132
Date:               2018-10-31 22:24   BIC:              22442.2081
No. Observations:   23918              Log-Likelihood:   -11105.   
Df Model:           22                 LL-Null:          -12615.   
Df Residuals:       23895              LLR p-value:      0.0000    
Converged:          1.0000             Scale:            1.0000    
No. Iterations:     7.0000                                         
--------------------------------------------------------------------
                Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
--------------------------------------------------------------------
LIMIT_BAL      -0.0000    0.0000   -4.9466  0.0000  -0.0000  -0.0000
SEX            -0.2043    0.0298   -6

In [8]:
%%time
# Scikit-Learn LR with regularization

param_grid = {'C': [0.01, 0.1, 0.5, 1.0], 'penalty': ['l1', 'l2']}

lrclf = LogisticRegression()
gs_clf = GridSearchCV(lrclf, param_grid, n_jobs=20, cv=5, scoring='roc_auc')

gs_clf.fit(train[X], train[y])

best_score_idx = np.argmax(gs_clf.cv_results_['mean_test_score'])
print("Best AUC on Test:",gs_clf.cv_results_['mean_test_score'][best_score_idx])
print("Best params:",gs_clf.cv_results_['params'][best_score_idx])

_c = gs_clf.cv_results_['params'][best_score_idx]['C']
_penalty = gs_clf.cv_results_['params'][best_score_idx]['penalty']

# Fit the model with best parameters of CV on TRAIN
lr = LogisticRegression(C = _c, penalty = _penalty)

Best AUC on Test: 0.7204884023880135
Best params: {'C': 0.01, 'penalty': 'l1'}
CPU times: user 1.04 s, sys: 219 ms, total: 1.26 s
Wall time: 6.88 s


In [10]:
lr.fit(train[X], train[y])

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
# Print the coefficients
intercept = lr.intercept_[0]
coeffs = lr.coef_[0]
print("Feature coefficients:")
print("INTERCEPT", "\t", intercept)
for f in range(len(coeffs)):
    print(X[f], "\t", coeffs[f])

Feature coefficients:
INTERCEPT 	 0.0
LIMIT_BAL 	 -1.0013382420863051e-06
SEX 	 -0.16472958804743493
EDUCATION 	 -0.1077347704416625
MARRIAGE 	 -0.2674460433191964
AGE 	 -0.0022395539019207417
PAY_0 	 0.5478926154847454
PAY_2 	 0.07351101990992452
PAY_3 	 0.06529383793142306
PAY_4 	 0.021857128626976037
PAY_5 	 0.03658127305060679
PAY_6 	 0.0
BILL_AMT1 	 -5.5960668555392926e-06
BILL_AMT2 	 2.7040431703382926e-06
BILL_AMT3 	 8.306319373086029e-07
BILL_AMT4 	 -7.659907725280922e-08
BILL_AMT5 	 8.516417767037056e-07
BILL_AMT6 	 6.863244349585328e-07
PAY_AMT1 	 -1.607410319606989e-05
PAY_AMT2 	 -1.1710777260131233e-05
PAY_AMT3 	 -3.066393819481443e-06
PAY_AMT4 	 -3.142093897919104e-06
PAY_AMT5 	 -3.4453520182428574e-06
PAY_AMT6 	 -1.7664554388054871e-06


In [12]:
preds = lr.predict(test[X])
probs = lr.predict_proba(test[X])

print(classification_report(test[y],preds))
print(confusion_matrix(test[y],preds))

#fpr, tpr, thresholds = roc_curve(test[y], probs[:,1], pos_label=1)
#print("AUC:", auc(fpr, tpr))

print("AUC:", roc_auc_score(test[y], probs[:, 1]))

             precision    recall  f1-score   support

          0       0.82      0.97      0.89      4718
          1       0.71      0.24      0.36      1364

avg / total       0.79      0.81      0.77      6082

[[4584  134]
 [1030  334]]
AUC: 0.7321333782518812


In [13]:
record_ind = 0

print("True Label:", test[y].iloc[[record_ind]])
print("Predicted Label:", preds[record_ind])
print("Prediction probability for default:", probs[record_ind][1])
print("Record to score:")
test[X].iloc[[record_ind]]

True Label: 0    1
Name: DEFAULT_NEXT_MONTH, dtype: int64
Predicted Label: 1
Prediction probability for default: 0.5437223070287793
Record to score:


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0


In [14]:
def get_reason_codes(data, ind, lr):
    probs = lr.predict_proba(data.iloc[[ind]])
    print("Prediction probability for default:", probs[0,1])
    partials = np.multiply(data.iloc[[ind]].values, lr.coef_)
    #print("Partals: ",partials)
    #print("Partials in desc order indices: ", (-partials).argsort())

    top_idx = (-partials).argsort()[0][:3]
    #print("\nTop reason code indices", top_idx)

    i = 0
    for f in top_idx:
        i+=1
        print("ReasonCode",i,": ", X[f], "\t", partials[0][f])
        

get_reason_codes(test[X], record_ind, lr)

Prediction probability for default: 0.5437223070287793
ReasonCode 1 :  PAY_0 	 1.0957852309694909
ReasonCode 2 :  PAY_2 	 0.14702203981984904
ReasonCode 3 :  BILL_AMT2 	 0.008387941914389383


### But, we can improve performance substantially by using a non-linear classifier like XGBoost

In [15]:
%%time

param_grid = {'max_depth': [3, 5], 'learning_rate': [0.1, 0.6], 'n_estimators': [100, 500], 'reg_alpha': [0.1, 1], 'reg_lambda': [0.1, 1]}

gbcclf = xgb.XGBClassifier(tree_method='hist', subsample=0.9, colsample_bytree=0.9)
gs_clf = GridSearchCV(gbcclf, param_grid, n_jobs=20, cv=5, scoring='roc_auc')

gs_clf.fit(train[X], train[y])

CPU times: user 2.31 s, sys: 317 ms, total: 2.62 s
Wall time: 35 s


In [16]:
best_score_idx = np.argmax(gs_clf.cv_results_['mean_test_score'])
print("Best AUC on Test:",gs_clf.cv_results_['mean_test_score'][best_score_idx])
print("Best params:",gs_clf.cv_results_['params'][best_score_idx])

_n_estimators = gs_clf.cv_results_['params'][best_score_idx]['n_estimators']
_max_depth = gs_clf.cv_results_['params'][best_score_idx]['max_depth']
_learning_rate = gs_clf.cv_results_['params'][best_score_idx]['learning_rate']
_reg_alpha = gs_clf.cv_results_['params'][best_score_idx]['reg_alpha']
_reg_lambda = gs_clf.cv_results_['params'][best_score_idx]['reg_lambda']

# Fit the model with best parameters of CV on TRAIN
xg = xgb.XGBClassifier(n_estimators = _n_estimators, learning_rate = _learning_rate, max_depth= _max_depth, reg_alpha = _reg_alpha, reg_lambda = _reg_lambda)


Best AUC on Test: 0.7799529070370025
Best params: {'reg_alpha': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_lambda': 1, 'learning_rate': 0.1}


In [17]:
%%time
xg.fit(train[X], train[y])

CPU times: user 1.88 s, sys: 347 µs, total: 1.89 s
Wall time: 1.88 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
# evaluate on TEST
preds = xg.predict(test[X])
probs = xg.predict_proba(test[X])

print(classification_report(test[y],preds))
print(confusion_matrix(test[y],preds))

#fpr, tpr, thresholds = roc_curve(test[y], probs[:,1], pos_label=1)
#print("AUC:", auc(fpr, tpr))
print("AUC:", roc_auc_score(test[y], probs[:, 1]))

importances = xg.feature_importances_
indices = np.argsort(importances)[::-1][:20]

# Print the feature ranking
print("Feature ranking:")
for f in range(10):
    print("%d. feature %s \t(%f)" % (f + 1, train[X].columns[indices[f]], importances[indices[f]]))

             precision    recall  f1-score   support

          0       0.84      0.95      0.89      4718
          1       0.68      0.38      0.48      1364

avg / total       0.80      0.82      0.80      6082

[[4478  240]
 [ 852  512]]
AUC: 0.782267077232139
Feature ranking:
1. feature BILL_AMT1 	(0.111272)
2. feature PAY_0 	(0.079480)
3. feature PAY_AMT3 	(0.076590)
4. feature LIMIT_BAL 	(0.073699)
5. feature PAY_AMT1 	(0.072254)
6. feature PAY_AMT2 	(0.052023)
7. feature BILL_AMT3 	(0.047688)
8. feature PAY_AMT5 	(0.047688)
9. feature AGE 	(0.043353)
10. feature BILL_AMT2 	(0.039017)


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### The above feature importance is on a global XGBoost level and isn't useful for explaining individual predictions. 